Below is the K-Means analysis for the IBM Applied Data Science Capstone. This Notebook accompanies the the final project report which is provided in a separate file.

* Install dependencies requiring pip

In [1]:
pip install BeautifulSoup4 -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geocoder -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lxml -q

Note: you may need to restart the kernel to use updated packages.


In [4]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: failed

CondaHTTPError: HTTP 000 CONNECTION FAILED for url <https://repo.anaconda.com/pkgs/free/linux-64/repodata.json.bz2>
Elapsed: -

An HTTP error occurred when trying to retrieve this URL.
HTTP errors are often intermittent, and a simple retry will get you on your way.

If your current network has https://www.anaconda.com blocked, please file
a support request with your network engineering team.

SSLError(MaxRetryError('HTTPSConnectionPool(host=\'repo.anaconda.com\', port=443): Max retries exceeded with url: /pkgs/free/linux-64/repodata.json.bz2 (Caused by SSLError("Can\'t connect to HTTPS URL because the SSL module is not available."))'))




* Import all required packages

In [14]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

from datetime import datetime

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn import preprocessing

import folium # map rendering library

print("Libraries imported.")

ModuleNotFoundError: No module named 'geopy'

* Define FourSquare Credentials and Version

In [8]:
CLIENT_ID = 'SZZA5NOLNSMV4XLJAEVQXUWB2A3K0YDOAS3KKBCQ2RTUPJT0' # your Foursquare ID
CLIENT_SECRET = 'KOLQQCRIG4WZMMAYQKHDA2KN1LFB4PZTH3JQ3Q2IGERW4UJR' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SZZA5NOLNSMV4XLJAEVQXUWB2A3K0YDOAS3KKBCQ2RTUPJT0
CLIENT_SECRET:KOLQQCRIG4WZMMAYQKHDA2KN1LFB4PZTH3JQ3Q2IGERW4UJR


* Get Data - Portland neighborhood names sourced from a csv file

In [9]:
hoods = pd.read_csv("portland_neighborhoods.csv")
print(hoods.shape)
hoods.head(3)

(109, 1)


,Neighborhood
0,Arlington Heights
1,Forest Park
2,Goose Hollow


* Get geographical coordinates for the neighborhoods


In [10]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Portland, Oregon'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [ ]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [get_latlng(neighborhood) for neighborhood in hoods["Neighborhood"].tolist()]
print(len(coords))
coords[:3]

In [ ]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [ ]:
# merge the coordinates into the original dataframe
hoods['Latitude'] = df_coords['Latitude']
hoods['Longitude'] = df_coords['Longitude']

In [ ]:
# check the neighborhoods and the coordinates
print(hoods.shape)
hoods.head(3)

In [ ]:
# save the DataFrame as CSV file
hoods.to_csv("hoods.csv", index=False)

* Create a map centered on Portland, OR with neighborhoods superimposed on top

In [ ]:
# get the coordinates of Portland
address = 'Portland, Oregon'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Portland, Oregon {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Portland using latitude and longitude values
portland_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(hoods['Latitude'], hoods['Longitude'], hoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(portland_map)

portland_map

In [ ]:
# save the map as HTML file
portland_map.save('portland_map.html')

In [ ]:
print(lat, lng)
hoods.head()
hoods.shape

* Use FourSquare API to explore the neighborhoods

In [ ]:
radius = 1000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(hoods['Latitude'], hoods['Longitude'], hoods['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [ ]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhoods', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(3)

* Count how many venues for each neighborhood

In [ ]:
venue_count = venues_df.groupby(["Neighborhoods"]).count()
print(venue_count.shape)
venue_count['VenueName'][:3]

* Find the number of unique categories within the venues

In [ ]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

In [ ]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:20]

* Analyze the Neighborhoods

In [ ]:
# one hot encoding
pdx_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
pdx_onehot['Neighborhoods'] = venues_df['Neighborhoods'] 

# move neighborhood column to the first column
fixed_columns = [pdx_onehot.columns[-1]] + list(pdx_onehot.columns[:-1])
pdx_onehot = pdx_onehot[fixed_columns]

print(pdx_onehot.shape)
pdx_onehot.head(3)

In [ ]:
#pdx_grouped = pdx_onehot.groupby(["Neighborhoods"]).count().reset_index()
pdx_grouped = pdx_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(pdx_grouped.shape)
pdx_grouped.head(3)

* Let's use boxplots to evaluate the distributions of the different features from the Foursquare App

In [ ]:
boxplotsa = pdx_grouped.drop(['Zoo Exhibit', 'Trail', 'Park','Nature Preserve', 'Hotel', 'Home Service', 'Golf Course', 'Garden', 'Forest', 'Dog Run', 'Disc Golf'], axis=1)
boxplotsa.boxplot(vert=False, figsize=(10,100))

In [ ]:
boxplotsb = pdx_grouped[['Zoo Exhibit', 'Trail', 'Park','Nature Preserve', 'Hotel', 'Home Service', 'Golf Course', 'Garden', 'Forest', 'Dog Run', 'Disc Golf']]
boxplotsb.boxplot(vert=False, figsize=(10,3))

*Import Portland Demographics by Neighborhood Data

In [ ]:
pd.read_html(requests.get("https://www.pdxmonthly.com/articles/2017/3/22/portland-neighborhoods-by-the-numbers-2017-the-city").content)[-1].to_csv("portland_data.csv")

* Create a dataframe for the Portland neighborhood demographics and real estate data

In [ ]:
pdx_data = pd.read_csv("portland_data.csv")
print(pdx_data.shape)
pdx_data.head(3)

* Get rid of columns that are redundant or irrelevant to the analysis

In [ ]:
no_info_columns = ["Unnamed: 0", "Adjusted population", "Minutes by car to downtown (est.)", "Parks and natural areas (Acres)", "Average home sale price ($)", "1-year median price change (2015-2016) (%)", "Distressed property sales (%)", "Married (except separated) (%)", "Divorced (%)", "Widowed (%)", "Separated (%)", "Never Married (%)", "Aggravated assault", "Arson", "Burglary", "Homicide", "Larceny", "Rape", "Robbery", "Vehicle thefts", "Nonviolent crimes", "Violent crimes", "Crimes per 1,000 residents", "Miles of bike routes (lanes, boulevards, multiuse paths)", "Miles of bike routes (all) per square mile", "Number of Max/streetcar lines", "Number of TriMet bus lines"]
pdx_data_clean = pdx_data.drop(no_info_columns, axis=1)
pdx_data_clean.shape

In [ ]:
pdx_data_clean.tail(3)

* There's a "Total" column buried in the dataframe from the original source, get rid of that.

In [ ]:
pdx_data_clean = pdx_data_clean.drop([95])
print(pdx_data_clean.shape)
pdx_data_clean.tail(3)

* "Year built (avg.)" column needs to be addressed because it is not meaningful to compare numbers like "1998" and "2010", the difference is miniscule. However, it is meaningful to compare the homes based on age, so create a new column based on a calculation of the building age using the current year and then drop the original column from the dataframe.

In [ ]:
pdx_data_clean = pdx_data_clean.drop("Year built (avg.)", axis=1)
pdx_data_clean.head(3)

* Let's use boxplots to evaluate the distributions of the different features from the Real Estate / Demographic data. Population moved from abroad...', 'Population in different state...', and 'Population in different country...' don't add any value as the distribution is very narrow, we will remove those from the analysis.

In [ ]:
boxplots1 = pdx_data_clean.drop(['Median age', 'Number of transit lines (bus/MAX/streetcar)', 'Days on market (avg.)', '5-year median price change (2012-2016) (%)', 'Walk score', 'Homes sold in 2016 (#)', 'Average cost per square foot ($)', 'Median household income ($)', 'Median home sale price ($)', 'Renters - median monthly housing expenses ($)', 'Adjusted population density (people per sq. mi., excluding parks and industrial tracts)'], axis=1)
boxplots1.boxplot(vert=False, figsize=(10,6))

In [ ]:
boxplots2 = pdx_data_clean[['Homes sold in 2016 (#)']]
boxplots2.boxplot(vert=False, figsize=(10, 1))

* Merge the two datasets, Foursquare (pdx_grouped) and the real estate data from Portland Monthly (pdx_data_clean)

In [ ]:
boxplots3 = pdx_data_clean[['Median home sale price ($)']]
boxplots3.boxplot(vert=False, figsize=(10,1))

In [ ]:
boxplots5 = pdx_data_clean[['Renters - median monthly housing expenses ($)']]
boxplots5.boxplot(vert=False, figsize=(10, 1))

In [ ]:
boxplots6 = pdx_data_clean[['Walk score']]
boxplots6.boxplot(vert=False, figsize=(10, 1))

In [ ]:
boxplots7 = pdx_data_clean[['5-year median price change (2012-2016) (%)']]
boxplots7.boxplot(vert=False, figsize=(10, 1))

In [ ]:
boxplots8 = pdx_data_clean[['Days on market (avg.)']]
boxplots8.boxplot(vert=False, figsize=(10, 1))

In [ ]:
boxplots9 = pdx_data_clean[['Average cost per square foot ($)']]
boxplots9.boxplot(vert=False, figsize=(10, 1))

In [ ]:
boxplots11 = pdx_data_clean[['Number of transit lines (bus/MAX/streetcar)']]
boxplots11.boxplot(vert=False, figsize=(10, 1))

In [ ]:
boxplots12 = pdx_data_clean[['Adjusted population density (people per sq. mi., excluding parks and industrial tracts)']]
boxplots12.boxplot(vert=False, figsize=(10, 1))
boxplots13 = pdx_data_clean[['Median household income ($)']]

In [ ]:
pdx_grouped.head(3)

In [ ]:
#Drop 3 columns based on narrow distribution in dataset
pdx_data_clean = pdx_data_clean.drop(['Population in different county last year (%)', 'Population moved from abroad last year (%)', 'Population in different state last year (%)'], axis=1)

#pdx_grouped pdx_data_clean
pdx_combined = pd.concat([pdx_grouped, pdx_data_clean], join='inner', axis=1)
print(pdx_combined.shape)
pdx_combined.head(3)

* Get rid of the string columns to prepare for normalization and clustering, but first make a copy of the column with "Neighborhoods" to be merged back in later so we can then merge the geospatial data for mapping.

In [ ]:
pdx_nbhd_column = pdx_combined[['Neighborhoods']]

In [ ]:
pdx_clustering = pdx_combined.drop(['Neighborhoods', 'Neighborhood'], 1)
pdx_column_names = pdx_clustering.columns

In [ ]:
pdx_clustering.describe()

* Normalize the data to improve the outputs of the algorithm by having every feature on a common scale

In [ ]:
# Create x, where x the 'scores' column's values as floats
x = pdx_clustering.values.astype(float)

# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)

# Run the normalizer on the dataframe
pdx_normalized = pd.DataFrame(x_scaled)
print(pdx_normalized.shape)
pdx_normalized.head(3)

In [ ]:
pdx_normalized.describe()

In [ ]:
# set number of clusters
kclusters = 12

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pdx_normalized)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
pdx_normalized.head(3)

* Add the clustering labels to a new dataframe, rename the columns to their original names, and insert the 'Neighborhood' column back into position index=0

In [ ]:
# create a new dataframe that includes the clusters.
pdx_merged = pdx_normalized.copy()

# add column names back in
pdx_merged.columns = pdx_column_names

pdx_merged.insert(0,'Neighborhood', pdx_nbhd_column)
#pdx_merged['Neighborhood'] = pdx_nbhd_column

# add clustering labels
pdx_merged["Cluster Labels"] = kmeans.labels_

In [ ]:
print(pdx_merged.shape)
pdx_merged.tail(3)

In [ ]:
# merge neighborhood geo data with the k-means data to add latitude/longitude for each neighborhood
pdx_merged = pdx_merged.join(hoods.set_index("Neighborhood"), on="Neighborhood")

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.jet(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pdx_merged['Latitude'], pdx_merged['Longitude'], pdx_merged['Neighborhood'], pdx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
summary_df = pdx_merged.groupby('Cluster Labels').mean()
summary_df = summary_df.drop('Latitude', axis=1)
summary_df = summary_df.drop('Longitude', axis=1)
print(summary_df.shape)
summary_df

* Check to see how many records for each Cluster Label by counting the records for any one of the columns

In [ ]:
pdx_merged_ct = pdx_merged.groupby('Cluster Labels').count()
pdx_merged_ct['ATM']

* Bar Charts showing top 20 category scores for each cluster. The data is sorted in ascending order by feature score so we will plot only the last 20 entries of the dataframe. These charts provide the most distinguishing characteristics attributed to the neighborhood cluster.

In [ ]:
#use a variable for setting the range for charting last 20 records
records_to_plot = -20

* Plot bar charts for each of the Clusters

In [ ]:
row = summary_df.iloc[0]
row.sort_values(axis = 0, ascending = True, inplace = True)
row = row[records_to_plot:]
#summary_df = summary_df.sort_values(row)
row.plot(kind='barh', figsize=(10,5))

In [ ]:
row = summary_df.iloc[1]
row.sort_values(axis = 0, ascending = True, inplace = True)
row = row[records_to_plot:]
#summary_df = summary_df.sort_values(row)
row.plot(kind='barh', figsize=(10,5))

In [ ]:
row = summary_df.iloc[2]
row.sort_values(axis = 0, ascending = True, inplace = True)
row = row[records_to_plot:]
#summary_df = summary_df.sort_values(row)
row.plot(kind='barh', figsize=(10,5))

In [ ]:
row = summary_df.iloc[3]
row.sort_values(axis = 0, ascending = True, inplace = True)
row = row[records_to_plot:]
#summary_df = summary_df.sort_values(row)
row.plot(kind='barh', figsize=(10,5))

In [ ]:
row = summary_df.iloc[4]
row.sort_values(axis = 0, ascending = True, inplace = True)
row = row[records_to_plot:]
#summary_df = summary_df.sort_values(row)
row.plot(kind='barh', figsize=(10,5))

In [ ]:
row = summary_df.iloc[5]
row.sort_values(axis = 0, ascending = True, inplace = True)
row = row[records_to_plot:]
#summary_df = summary_df.sort_values(row)
row.plot(kind='barh', figsize=(10,5))

In [ ]:
row = summary_df.iloc[6]
row.sort_values(axis = 0, ascending = True, inplace = True)
row = row[records_to_plot:]
#summary_df = summary_df.sort_values(row)
row.plot(kind='barh', figsize=(10,5))

In [ ]:
row = summary_df.iloc[7]
row.sort_values(axis = 0, ascending = True, inplace = True)
row = row[records_to_plot:]
#summary_df = summary_df.sort_values(row)
row.plot(kind='barh', figsize=(10,5))

In [ ]:
row = summary_df.iloc[8]
row.sort_values(axis = 0, ascending = True, inplace = True)
row = row[records_to_plot:]
#summary_df = summary_df.sort_values(row)
row.plot(kind='barh', figsize=(10,5))

In [ ]:
row = summary_df.iloc[9]
row.sort_values(axis = 0, ascending = True, inplace = True)
row = row[records_to_plot:]
#summary_df = summary_df.sort_values(row)
row.plot(kind='barh', figsize=(10,5))

In [ ]:
row = summary_df.iloc[10]
row.sort_values(axis = 0, ascending = True, inplace = True)
row = row[records_to_plot:]
#summary_df = summary_df.sort_values(row)
row.plot(kind='barh', figsize=(10,5))

In [ ]:
row = summary_df.iloc[11]
row.sort_values(axis = 0, ascending = True, inplace = True)
row = row[records_to_plot:]
#summary_df = summary_df.sort_values(row)
row.plot(kind='barh', figsize=(10,5))